<a href="https://colab.research.google.com/github/nicolasrojasv/nicolasrojasv.github.io/blob/main/ranking_cutoff_gini_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Times series inequality in professional tennis with ranking cut-off and comparing it with countries**

In [1]:
#Load libraries
import pandas as pd
import numpy as np
import altair as alt
import zipfile
import io
import requests

In [2]:
#Load ATP players activity data and ranking data
players_activity = pd.read_csv('https://raw.githubusercontent.com/nicolasrojasv/nicolasrojasv.github.io/refs/heads/main/Project/data/players_activity.csv')

In [3]:
#Check tables
display(players_activity.head())

,player_id,year,event_id,event_name,event_title,prize_raw,currency,prize_usd
0,sy30,2019,8332,M25 Claremont,M25 Claremont,260,$,260
1,sy30,2019,1592,M25 Harlingen,M25 Harlingen,730,$,730
2,sy30,2019,8315,M15 Cancun,M15 Cancun,258,$,258
3,sy30,2019,8308,M15 Cancun,M15 Cancun,258,$,258
4,sy30,2019,8306,M15 Cancun,M15 Cancun,753,$,753


In [4]:
#Prizes not awarded in tournaments are excluded from the analysis.
#This is solely to ensure consistency with the information displayed on the website.
exclude = ["500 Bonus Pool", "1000 Bonus Pool", "Profit Sharing", "Profit Share", "Bonus Prize Money"]
players_activity_filtered = players_activity[~players_activity["event_name"].isin(exclude)]

In [5]:
#Calculate the prize money by player and year
players_prize_by_year = players_activity_filtered.groupby(["player_id", "year"])["prize_usd"].sum().reset_index()
players_prize_by_year.tail(10)

,player_id,year,prize_usd
42995,z450,2017,1470
42996,z450,2018,2274
42997,z450,2019,1666
42998,z452,2016,812
42999,z452,2017,1704
43000,z452,2018,1804
43001,z457,2015,182
43002,z457,2016,104
43003,z460,2014,414
43004,z460,2015,516


In [6]:
#Sort the table by prize money and year
players_prize_by_year = players_prize_by_year.sort_values(by=['year', 'prize_usd'], ascending=False)
players_prize_by_year.tail(10)

,player_id,year,prize_usd
10458,e121,1991,5366
34718,s535,1991,1040
23876,m343,1991,520
10457,e121,1990,1820
23875,m343,1990,1300
28153,o111,1989,212
23874,m343,1988,1194
23873,m343,1987,540
23872,m343,1985,680
3732,bf71,1956,0


In [7]:
#Check the unique year in the table
players_prize_by_year['year'].unique()

array([2025, 2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015,
       2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004,
       2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993,
       1992, 1991, 1990, 1989, 1988, 1987, 1985, 1956])

In [8]:
#The table has activity from 1956 (possible an error when extracting, but the price is 0) to 2025.
#For simplicity the project start at 2014.
players_prize_by_year = players_prize_by_year[players_prize_by_year['year'] >= 2014]

In [9]:
#Generate a column with the rank of the player according to the price money by each year
players_prize_by_year['rank'] = players_prize_by_year.groupby('year')['prize_usd'].rank(ascending=False, method='min')
players_prize_by_year.head(10)

,player_id,year,prize_usd,rank
33544,s0ag,2025,19114398,1.0
239,a0e2,2025,18803427,2.0
42830,z355,2025,5976269,3.0
12363,fb98,2025,5456536,4.0
1743,ag37,2025,5228696,5.0
10282,dh58,2025,5192228,6.0
8809,d643,2025,5127247,7.0
23145,m0ej,2025,4682629,8.0
34370,s0s1,2025,4600654,9.0
8232,d0co,2025,3421705,10.0


In [10]:
# Grid to generate lorenz curve by ranking
TOP_N_GRID = [50, 100, 150, 200, 300, 500, 1000, 2000]
TOP_N_ALL = 99999  # sentinel value for "All ranked players"
N_POINTS = 101     # points along the Lorenz curve (0..1)

In [11]:
#Lorenz curve with 101 points
def lorenz_from_values(values: pd.Series, n_points: int = 101) -> pd.DataFrame:
    """
    Build a Lorenz curve from non-negative values.
    Returns a dataframe with p (population share) and lorenz (income share).
    """
    v = values.dropna().astype(float).clip(lower=0).to_numpy()
    if v.size == 0:
        return pd.DataFrame({"p": np.linspace(0, 1, n_points), "lorenz": np.nan})

    v.sort()
    cum_v = np.cumsum(v)
    total = cum_v[-1]

    # If total income is zero, Lorenz curve is undefined; return zeros
    if total == 0:
        return pd.DataFrame({"p": np.linspace(0, 1, n_points), "lorenz": 0.0})

    # Empirical Lorenz points at each observation
    p_obs = np.arange(1, v.size + 1) / v.size
    L_obs = cum_v / total

    # Add the origin (0,0)
    p_obs = np.insert(p_obs, 0, 0.0)
    L_obs = np.insert(L_obs, 0, 0.0)

    # Interpolate to a fixed grid for plotting
    p_grid = np.linspace(0, 1, n_points)
    L_grid = np.interp(p_grid, p_obs, L_obs)

    return pd.DataFrame({"p": p_grid, "lorenz": L_grid})


In [12]:
#Function for calculate Gini
def gini_from_lorenz(p: np.ndarray, L: np.ndarray) -> float:
    """
    Compute Gini from a Lorenz curve using numerical integration:
    G = 1 - 2 * area_under_L(p)
    """
    area = np.trapz(L, p)
    return float(1 - 2 * area)

In [13]:
#Function that generate the dataframe with lorenz curve for the grid defined
def build_lorenz_panel(df_players: pd.DataFrame,
                       year_col: str = "year",
                       rank_col: str = "rank",
                       income_col: str = "prize_usd") -> pd.DataFrame:
    """
    Precompute Lorenz curves for each year and each top_n in the grid.
    Expects one row per player-year.
    """
    out = []

    # Build grid including the sentinel for "All"
    grid = TOP_N_GRID + [TOP_N_ALL]

    for yr, d in df_players.groupby(year_col):
        d = d.copy()

        # Ensure rank and income are numeric
        d[rank_col] = pd.to_numeric(d[rank_col], errors="coerce")
        d[income_col] = pd.to_numeric(d[income_col], errors="coerce")

        # Drop missing ranks
        d = d.dropna(subset=[rank_col])

        # Sort by rank ascending (1 is best)
        d = d.sort_values(rank_col)

        for top_n in grid:
            # Select top N or all
            if top_n == TOP_N_ALL:
                d_sel = d
            else:
                d_sel = d.head(top_n)

            # Build Lorenz curve from income values
            curve = lorenz_from_values(d_sel[income_col], n_points=N_POINTS)

            # Compute Gini for this curve
            gini_val = gini_from_lorenz(curve["p"].to_numpy(), curve["lorenz"].to_numpy())

            curve["year"] = int(yr)
            curve["top_n"] = int(top_n)
            curve["gini"] = gini_val

            out.append(curve)

    return pd.concat(out, ignore_index=True)

In [14]:
# df_players should be your player-year dataset with columns: year, rank, prize_money
lorenz_panel = build_lorenz_panel(players_prize_by_year)

/tmp/ipython-input-2385084487.py:7: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  area = np.trapz(L, p)
/tmp/ipython-input-2385084487.py:7: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  area = np.trapz(L, p)
/tmp/ipython-input-2385084487.py:7: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  area = np.trapz(L, p)
/tmp/ipython-input-2385084487.py:7: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  area = np.trapz(L, p)
/tmp/ipython-input-2385084487.py:7: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  area = np.trapz(L, p)
/tmp/ipython-input-2

In [15]:
lorenz_panel[lorenz_panel['year'] == 2025]

,p,lorenz,year,top_n,gini
9999,0.00,0.000000,2025,50,0.389292
10000,0.01,0.004307,2025,50,0.389292
10001,0.02,0.008613,2025,50,0.389292
10002,0.03,0.013015,2025,50,0.389292
10003,0.04,0.017417,2025,50,0.389292
...,...,...,...,...,...
10903,0.96,0.255417,2025,99999,0.922166
10904,0.97,0.330641,2025,99999,0.922166
10905,0.98,0.431007,2025,99999,0.922166
10906,0.99,0.576252,2025,99999,0.922166


In [16]:
#Save the lorenz panel data set
lorenz_panel.to_csv("lorenz_curves_by_year_topn.csv", index=False)

In [17]:
#Function to calculate gini index
def gini_from_values(values: pd.Series) -> float:
    """
    Compute the Gini coefficient from non-negative values
    and round it to three decimal places.
    """
    v = values.dropna().astype(float).clip(lower=0).to_numpy()
    if v.size == 0:
        return np.nan

    # If all values are zero, inequality is undefined; return 0 by convention
    if np.all(v == 0):
        return 0.0

    v.sort()
    n = v.size
    cumv = np.cumsum(v)
    total = cumv[-1]

    g = (n + 1 - 2 * np.sum(cumv) / total) / n

    # Round Gini to three decimal places
    return round(float(g), 3)

#Function to retrieved gini index from the ranking cutoff
def build_gini_timeseries(df_players: pd.DataFrame,
                          year_col: str = "year",
                          rank_col: str = "rank",
                          income_col: str = "prize_usd") -> pd.DataFrame:
    """
    Precompute Gini time series for each year and each top_n in the grid.
    Expects one row per player-year.
    """
    out = []
    grid = TOP_N_GRID + [TOP_N_ALL]

    for yr, d in df_players.groupby(year_col):
        d = d.copy()
        d[rank_col] = pd.to_numeric(d[rank_col], errors="coerce")
        d[income_col] = pd.to_numeric(d[income_col], errors="coerce")

        # Keep rows with valid ranks and sort by rank (1 = best)
        d = d.dropna(subset=[rank_col]).sort_values(rank_col)

        for top_n in grid:
            d_sel = d if top_n == TOP_N_ALL else d.head(top_n)
            gini_val = gini_from_values(d_sel[income_col])

            out.append({
                "year": int(yr),
                "top_n": int(top_n),
                "gini": gini_val
            })

    return pd.DataFrame(out)

In [18]:
gini_ts = build_gini_timeseries(players_prize_by_year)
gini_ts[gini_ts['year'] == 2025]

,year,top_n,gini
99,2025,50,0.389
100,2025,100,0.422
101,2025,150,0.476
102,2025,200,0.529
103,2025,300,0.613
104,2025,500,0.725
105,2025,1000,0.832
106,2025,2000,0.904
107,2025,99999,0.924


In [19]:
#Create two columns in the gini_ts with country name and country code and assign ATP TOUR and ATP to them
gini_ts["country_name"] = "ATP Tour"
gini_ts["country_code"] = "ATP"
gini_ts.head()

,year,top_n,gini,country_name,country_code
0,2014,50,0.481,ATP Tour,ATP
1,2014,100,0.522,ATP Tour,ATP
2,2014,150,0.573,ATP Tour,ATP
3,2014,200,0.624,ATP Tour,ATP
4,2014,300,0.703,ATP Tour,ATP


In [20]:
#Load Countries' Gini Index from the world Bank
url = "https://api.worldbank.org/v2/en/indicator/SI.POV.GINI?downloadformat=csv"

# Download zip in memory
response = requests.get(url)
zip_bytes = io.BytesIO(response.content)

with zipfile.ZipFile(zip_bytes) as z:
    # Identified the file which start with "API"
    api_file = [f for f in z.namelist() if f.startswith("API") and f.endswith(".csv")][0]

    # Read the csv file
    country_gini = pd.read_csv(
        z.open(api_file),
        skiprows=4,
        header=0,
    )

# Standardize column names
country_gini.columns = (
    country_gini.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)

In [21]:
# Reshape data from wide to long format (years as rows instead of columns)
country_gini = country_gini.melt(
    id_vars=["country_name", "country_code", "indicator_name", "indicator_code"],
    var_name="year",
    value_name="gini"
)

# Convert year column to numeric, coercing invalid values to NaN
country_gini["year"] = pd.to_numeric(country_gini["year"], errors="coerce")

In [22]:
# Sort to ensure forward fill uses the last available previous year within each country
country_gini = country_gini.sort_values(["country_code", "year"])

# Forward-fill missing Gini values within each country (carry last observation forward)
country_gini["gini"] = country_gini.groupby("country_code")["gini"].ffill()

In [23]:
#keep the data from 2014 onwards
country_gini = country_gini[country_gini["year"] >= 2014]

In [24]:
#Use the same data from 2024 for 2025
country_gini_2025 = country_gini[country_gini["year"] == 2024].copy() # Explicitly create a copy
country_gini_2025["year"] = 2025

In [25]:
# Concatenate the 2025 data (derived from 2024) with the main country_gini DataFrame
country_gini = pd.concat([country_gini, country_gini_2025], ignore_index=True)

# Display the tail of the updated DataFrame to show the newly added 2025 entries
display(country_gini.tail())

,country_name,country_code,indicator_name,indicator_code,year,gini
3187,Kosovo,XKX,Gini index,SI.POV.GINI,2025.0,38.3
3188,"Yemen, Rep.",YEM,Gini index,SI.POV.GINI,2025.0,36.7
3189,South Africa,ZAF,Gini index,SI.POV.GINI,2025.0,63.0
3190,Zambia,ZMB,Gini index,SI.POV.GINI,2025.0,51.5
3191,Zimbabwe,ZWE,Gini index,SI.POV.GINI,2025.0,50.3


In [26]:
#Drop columns indicator_name and indicator_code
country_gini = country_gini.drop(columns=["indicator_name", "indicator_code"])


In [27]:
# Convert country Gini from 0–100 to 0–1
country_gini["gini"] = pd.to_numeric(country_gini["gini"], errors="coerce") / 100

In [28]:
# Ensure year is integer in both datasets
df_atp = gini_ts.copy()
df_countries = country_gini.copy()
df_atp["year"] = pd.to_numeric(df_atp["year"], errors="coerce").astype("Int64")
df_countries["year"] = pd.to_numeric(df_countries["year"], errors="coerce").astype("Int64")

In [29]:
# Create a dataframe with all top_n values
top_n_values = TOP_N_GRID + [TOP_N_ALL]
df_topn = pd.DataFrame({"top_n": top_n_values})

# Replicate every country-year row for every top_n (cross join)
df_countries_expanded = df_countries.merge(df_topn, how="cross")

# Put columns in the same order before concatenating
df_countries_expanded = df_countries_expanded[["year", "top_n", "gini", "country_name", "country_code"]]
df_atp_aligned = df_atp[["year", "top_n", "gini", "country_name", "country_code"]]

# Concatenate both datasets
df_combined = pd.concat([df_atp_aligned, df_countries_expanded], ignore_index=True)

In [30]:
#Save dataset with ATP Tour and countries Gini
df_combined.to_csv("gini_by_year_topn.csv", index=False)